In [1]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 4.0 MB/s eta 0:00:00a 0:00:01


In [2]:
import duckdb

In [29]:
duckdb.sql("SELECT * FROM 'train.csv' LIMIT 5").df()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [28]:
duckdb.sql(""" 
SELECT *
FROM 'train.csv'
WHERE Pclass=3 AND Age<18;
""").df()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,None,S
1,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
2,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,None,S
3,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,None,Q
4,23,1,3,"McGowan, Miss. Anna ""Annie""",female,15.0,0,0,330923,8.0292,None,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
73,845,0,3,"Culumovic, Mr. Jeso",male,17.0,0,0,315090,8.6625,None,S
74,851,0,3,"Andersson, Master. Sigvard Harald Elias",male,4.0,4,2,347082,31.2750,None,S
75,853,0,3,"Boulos, Miss. Nourelain",female,9.0,1,1,2678,15.2458,None,C
76,870,1,3,"Johnson, Master. Harold Theodor",male,4.0,1,1,347742,11.1333,None,S


In [30]:
duckdb.sql("""
Select Pclass,
count(*) as Total_passengers,
sum(Survived) as Total_survived,
Round(sum(Survived) * 1.0 / count(*),2) as Survival_rate
from 'train.csv'
group by Pclass
order by Survival_rate Desc;
""").df()


,Pclass,Total_passengers,Total_survived,Survival_rate
0,1,216,136.0,0.63
1,2,184,87.0,0.47
2,3,491,119.0,0.24


In [26]:
duckdb.sql("""
CREATE OR REPLACE TABLE titanic AS
SELECT * FROM read_csv_auto('train.csv')
""")

In [27]:
duckdb.sql("""
SELECT *
FROM titanic
WHERE Age IS NOT NULL
  AND Fare > 30
LIMIT 5;
""").df()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
3,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,None,C
4,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,None,S


In [31]:
duckdb.sql("""
CREATE OR REPLACE TABLE titanic AS
SELECT *,
       (SibSp + Parch + 1) AS family_size,
       ROUND(Fare / NULLIF((SibSp + Parch + 1), 0), 2) AS fare_per_person,
       CASE 
           WHEN Age < 16 THEN 1
           ELSE 0
       END AS is_child
FROM titanic
""")

In [33]:
duckdb.sql("""
SELECT PassengerId, family_size, fare_per_person, is_child 
FROM titanic
LIMIT 5""").df()


,PassengerId,family_size,fare_per_person,is_child
0,1,2,3.63,0
1,2,2,35.64,0
2,3,1,7.93,0
3,4,2,26.55,0
4,5,1,8.05,0


In [34]:
duckdb.sql("""
SELECT
    Pclass,
    Sex,
    is_child,
    ROUND(AVG(Survived), 2) AS survival_rate,
    COUNT(*) AS passenger_count
FROM titanic
GROUP BY Pclass, Sex, is_child
ORDER BY Pclass, Sex, is_child;
""").df()

,Pclass,Sex,is_child,survival_rate,passenger_count
0,1,female,0,0.98,91
1,1,female,1,0.67,3
2,1,male,0,0.35,119
3,1,male,1,1.00,3
4,2,female,0,0.91,66
5,2,female,1,1.00,10
6,2,male,0,0.08,99
7,2,male,1,1.00,9
8,3,female,0,0.49,114
9,3,female,1,0.53,30


In [35]:
duckdb.sql("""
SELECT
    family_size,
    ROUND(AVG(Survived), 2) AS survival_rate,
    COUNT(*) AS passenger_count
FROM titanic
GROUP BY family_size
ORDER BY family_size;
""").df()

,family_size,survival_rate,passenger_count
0,1,0.30,537
1,2,0.55,161
2,3,0.58,102
3,4,0.72,29
4,5,0.20,15
5,6,0.14,22
6,7,0.33,12
7,8,0.00,6
8,11,0.00,7


In [36]:
duckdb.sql("""
SELECT
    CASE
        WHEN Fare < 10 THEN 'Low Fare'
        WHEN Fare BETWEEN 10 AND 50 THEN 'Medium Fare'
        ELSE 'High Fare'
    END AS fare_category,
    ROUND(AVG(Survived), 2) AS survival_rate,
    COUNT(*) AS passenger_count
FROM titanic
GROUP BY fare_category
ORDER BY survival_rate DESC;
""").df()

,fare_category,survival_rate,passenger_count
0,High Fare,0.68,160
1,Medium Fare,0.42,395
2,Low Fare,0.20,336


In [40]:
duckdb.sql("""
SELECT
    CASE 
        WHEN Age < 13 THEN 'Child'
        WHEN Age BETWEEN 13 AND 19 THEN 'Teenager'
        WHEN Age BETWEEN 20 AND 59 THEN 'Adult'
        ELSE 'Senior'
    END as Age_group,
    ROUND(AVG(Survived),2) as survival_rate,
    COUNT (*) as passenger_count,
FROM titanic
WHERE Age IS NOT NULL
GROUP BY Age_group
ORDER BY survival_rate DESC; 
""").df()

,Age_group,survival_rate,passenger_count
0,Child,0.58,69
1,Teenager,0.41,95
2,Adult,0.39,524
3,Senior,0.27,26


In [44]:
duckdb.sql("""
SELECT
    CASE
        WHEN Fare <10 THEN 'Low Fare'
        WHEN Fare BETWEEN 10 AND 50 THEN 'Medium Fare'
        ELSE 'High'
        END as Fare_category,

    CASE 
        WHEN Age<13 THEN 'Child'
        WHEN Age BETWEEN 13 AND 19 THEN 'Teenager'
        WHEN Age BETWEEN 20 AND 59 THEN 'Adult'
        ELSE 'Senior'
        END as Age_group,
    ROUND(AVG(Survived),2) as survival_rate,
    COUNT(*) as passenger_count,
FROM titanic
WHERE Fare IS NOT NULL AND Age IS NOT NULL
GROUP BY Fare_category,Age_group
ORDER BY Fare_category,Age_group
""").df()

,Fare_category,Age_group,survival_rate,passenger_count
0,High,Adult,0.71,112
1,High,Child,0.75,4
2,High,Senior,0.57,7
3,High,Teenager,0.73,15
4,Low Fare,Adult,0.17,187
5,Low Fare,Child,1.00,1
6,Low Fare,Senior,0.20,5
7,Low Fare,Teenager,0.33,43
8,Medium Fare,Adult,0.41,225
9,Medium Fare,Child,0.56,64


In [46]:
median_age = duckdb.sql("SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Age) AS median_age FROM titanic WHERE Age IS NOT NULL").df().iloc[0, 0]

In [47]:
print(median_age)

28.0


In [51]:
duckdb.sql(f"""
CREATE OR REPLACE TABLE titanic_cleaned AS
SELECT *,
    CASE 
    WHEN Age is NULL THEN 1 
    ELSE 0
    END as age_was_missing,
    COALESCE(Age,{median_age}) AS age_filled
FROM titanic
""")
    

In [60]:
duckdb.sql(f"""
CREATE OR REPLACE VIEW titanic_features AS
SELECT 
    PassengerId,
    Survived,
    Pclass,
    Sex,
    COALESCE(Age,{median_age}) AS Age,
    Sibsp,
    Parch,
    (Sibsp+Parch+1) AS family_size,
    ROUND(Fare/NULLIF((Sibsp+Parch+1),0),2) AS fare_per_person,
    CASE
        WHEN COALESCE(Age,{median_age}) <16 THEN 1
        ELSE 0
        END AS is_child,
        Embarked
FROM titanic
""")

In [62]:
duckdb.sql(f"""
CREATE OR REPLACE VIEW titanic_features AS
    SELECT *, 
        (Sibsp+Parch+1) AS family_size,
        ROUND(Fare/NULLIF((Sibsp+Parch+1),0),2) AS fare_per_person,
        CASE
            WHEN Age<18 THEN 1
            ELSE 0
            END AS is_child,
            COALESCE(Age, {median_age}) AS imputed_age,
FROM titanic
""")


In [64]:
duckdb.sql("SELECT * FROM titanic_features LIMIT 5").df()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,family_size,fare_per_person,is_child,family_size_1,fare_per_person_1,is_child_1,imputed_age
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,2,3.63,0,2,3.63,0,22.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,35.64,0,2,35.64,0,38.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,1,7.93,0,1,7.93,0,26.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,26.55,0,2,26.55,0,35.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,1,8.05,0,1,8.05,0,35.0


In [65]:
duckdb.sql("""
SELECT Name, Age, Sex
FROM titanic_features
WHERE is_child = 1
LIMIT 10;
""").df()

,Name,Age,Sex
0,"Palsson, Master. Gosta Leonard",2.0,male
1,"Nasser, Mrs. Nicholas (Adele Achem)",14.0,female
2,"Sandstrom, Miss. Marguerite Rut",4.0,female
3,"Vestrom, Miss. Hulda Amanda Adolfina",14.0,female
4,"Rice, Master. Eugene",2.0,male
5,"McGowan, Miss. Anna ""Annie""",15.0,female
6,"Palsson, Miss. Torborg Danira",8.0,female
7,"Nicola-Yarred, Miss. Jamila",14.0,female
8,"Laroche, Miss. Simonne Marie Anne Andree",3.0,female
9,"Panula, Master. Juha Niilo",7.0,male


In [67]:
duckdb.sql("""
SELECT Name, Pclass, Fare
FROM titanic_features
WHERE Pclass=1 AND Fare >100
ORDER BY Fare DESC
LIMIT 10;
""").df()
 

,Name,Pclass,Fare
0,"Ward, Miss. Anna",1,512.3292
1,"Cardeza, Mr. Thomas Drake Martinez",1,512.3292
2,"Lesurer, Mr. Gustave J",1,512.3292
3,"Fortune, Mr. Charles Alexander",1,263.0000
4,"Fortune, Miss. Mabel Helen",1,263.0000
5,"Fortune, Miss. Alice Elizabeth",1,263.0000
6,"Fortune, Mr. Mark",1,263.0000
7,"Ryerson, Miss. Emily Borie",1,262.3750
8,"Ryerson, Miss. Susan Parker ""Suzette""",1,262.3750
9,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",1,247.5208


In [69]:
duckdb.sql("""
SELECT Name, Cabin
FROM titanic_features
WHERE Cabin IS NULL
LIMIT 10;
""")

┌───────────────────────────────────────────────────┬─────────┐
│                       Name                        │  Cabin  │
│                      varchar                      │ varchar │
├───────────────────────────────────────────────────┼─────────┤
│ Braund, Mr. Owen Harris                           │ NULL    │
│ Heikkinen, Miss. Laina                            │ NULL    │
│ Allen, Mr. William Henry                          │ NULL    │
│ Moran, Mr. James                                  │ NULL    │
│ Palsson, Master. Gosta Leonard                    │ NULL    │
│ Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg) │ NULL    │
│ Nasser, Mrs. Nicholas (Adele Achem)               │ NULL    │
│ Saundercock, Mr. William Henry                    │ NULL    │
│ Andersson, Mr. Anders Johan                       │ NULL    │
│ Vestrom, Miss. Hulda Amanda Adolfina              │ NULL    │
├───────────────────────────────────────────────────┴─────────┤
│ 10 rows                               

In [70]:
duckdb.sql("""
SELECT Pclass, ROUND(AVG(Fare),2) as avg_fare
FROM titanic_features
GROUP BY Pclass
ORDER BY Pclass DESC;
""").df()

,Pclass,avg_fare
0,3,13.68
1,2,20.66
2,1,84.15


In [107]:
duckdb.sql("""
SELECT SEX, ROUND(AVG(Survived) * 100,2) AS avg_survival_rate
FROM titanic_features
GROUP BY SEX;
""").df()

,Sex,avg_survival_rate
0,male,18.89
1,female,74.20


In [80]:
duckdb.sql("""
SELECT Embarked,COUNT(*) AS passenger_count
FROM titanic_features
GROUP BY Embarked
ORDER BY passenger_count DESC;
""").df()

,Embarked,passenger_count
0,S,644
1,C,168
2,Q,77
3,None,2


In [81]:
duckdb.sql("""
SELECT Embarked,COUNT(*) AS passenger_count
FROM titanic_features
GROUP BY Embarked
HAVING COUNT(*) >100
ORDER BY passenger_count DESC;
""").df()

,Embarked,passenger_count
0,S,644
1,C,168


In [86]:
duckdb.sql("""
SELECT SEX, ROUND(AVG(Survived) * 100,2) AS avg_survival_rate
FROM titanic_features
GROUP BY SEX
HAVING AVG(Survived) >0.5
ORDER BY avg_survival_rate DESC;
""").df()

,Sex,avg_survival_rate
0,female,74.2


In [88]:
duckdb.sql("""
WITH survival_by_port AS (
    SELECT Embarked,
        Count(*) AS total_passengers,
        ROUND(AVG(Survived) *100,2) AS survival_rate_percent
        FROM titanic_features
        GROUP BY Embarked
)
SELECT * 
FROM survival_by_port
WHERE total_passengers >100
ORDER BY survival_rate_percent DESC;
""").df()
        


,Embarked,total_passengers,survival_rate_percent
0,C,168,55.36
1,S,644,33.70


In [97]:
duckdb.sql("""
CREATE OR REPLACE TABLE port_info AS
SELECT * FROM (
VALUES
    ('S','Southampton','UK'),
    ('C','Cherbourg','France'),
    ('Q','Queenstown','Ireland'),
)AS t(Embarked,Port_name,Country)
""")


In [92]:
duckdb.sql("SELECT * FROM port_info").df()

,Embarked,Port_name,Country
0,S,Southampton,UK
1,C,Cherbourg,France
2,Q,Queenstown,Ireland


In [102]:
duckdb.sql("""
SELECT 
    tf.Name,
    tf.Embarked,
    pi.Port_name,
    pi.Country
FROM titanic_features AS tf
LEFT JOIN port_info AS pi
ON tf.Embarked=pi.Embarked
LIMIT 10;
""").df()

,Name,Embarked,Port_name,Country
0,"Braund, Mr. Owen Harris",S,Southampton,UK
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",C,Cherbourg,France
2,"Heikkinen, Miss. Laina",S,Southampton,UK
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",S,Southampton,UK
4,"Allen, Mr. William Henry",S,Southampton,UK
5,"Moran, Mr. James",Q,Queenstown,Ireland
6,"McCarthy, Mr. Timothy J",S,Southampton,UK
7,"Palsson, Master. Gosta Leonard",S,Southampton,UK
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",S,Southampton,UK
9,"Nasser, Mrs. Nicholas (Adele Achem)",C,Cherbourg,France


In [106]:
duckdb.sql("""
SELECT 
    tf.*,
    t.Port_name,
    t.Country
FROM titanic_features tf
LEFT JOIN(
    VALUES 
    ('S', 'Southampton', 'UK'),
    ('C', 'Cherbourg', 'France'),
    ('Q', 'Queenstown', 'Ireland')
) AS t(Embarked,Port_name,Country)
ON tf.Embarked=t.Embarked
LIMIT 5;
""").df()

        

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked,family_size,fare_per_person,is_child,family_size_1,fare_per_person_1,is_child_1,imputed_age,Port_name,Country
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,S,2,3.63,0,2,3.63,0,22.0,Southampton,UK
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,C,2,35.64,0,2,35.64,0,38.0,Cherbourg,France
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,S,1,7.93,0,1,7.93,0,26.0,Southampton,UK
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,S,2,26.55,0,2,26.55,0,35.0,Southampton,UK
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,S,1,8.05,0,1,8.05,0,35.0,Southampton,UK


In [111]:
duckdb.sql("""
COPY (
    SELECT * FROM titanic_features
)
TO 'titanic_cleaned.csv'
WITH (HEADER, DELIMITER ',')
""")

In [112]:
import pandas as pd
df = pd.read_csv("titanic_cleaned.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,family_size,fare_per_person,is_child,family_size_1,fare_per_person_1,is_child_1,imputed_age
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,3.63,0,2,3.63,0,22.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,35.64,0,2,35.64,0,38.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,7.93,0,1,7.93,0,26.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,26.55,0,2,26.55,0,35.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,8.05,0,1,8.05,0,35.0
